# Extract blocks of trials from the whole runs

In [1]:
import os
from os.path import join as opj
import numpy as np
import nibabel as nib
import pandas as pd
import re

In [9]:
INPUT_DIR = r"...\data\preprocessed\datasink\preproc"  # need to be set
OUTPUT_DIR = r"...\HW\data\trials\data"  # need to be set
LABEL_CSV = r"...\data\trials\labels.csv"  # need to be set

In [18]:
event_file = r"...\data\ds003548\task-emotionalfaces_run-1_events.tsv"   # need to be set

trialinfo = pd.read_table(event_file)
trialinfo.head()

trialinfo['onset'] = trialinfo['onset'] // 2
trialinfo['duration'] = trialinfo['duration'] // 2

trial_type_ids = {
    'neutral': 0,
    'happy': 1,
    'sad': 2,
    'angry': 3,
    'scrambled': 4,
    'blank': 5,
    'end': 6
}

trialinfo

,onset,duration,trial_type
0,0,15,blank
1,15,15,scrambled
2,30,15,happy
3,45,15,sad
4,60,15,angry
5,75,15,neutral
6,90,15,happy
7,105,15,sad
8,120,15,angry
9,135,15,neutral


In [ ]:
# Collect per subject, pre run fMRI files
fmri_pths = []

for subj_dir in os.listdir(INPUT_DIR):
    subj_dir_pth = opj(INPUT_DIR, subj_dir, 'task-emotionalfaces')
    for run_dir in os.listdir(subj_dir_pth):
        run_dir_pth = opj(subj_dir_pth, run_dir)
        for file_name in os.listdir(run_dir_pth):
            if os.path.splitext(file_name)[-1] == '.gz' and 'fwhm' in file_name:
                fmri_pths.append(
                    opj(run_dir_pth, file_name)
                )
fmri_pths

In [19]:
subjects = []
runs = []
ext_fmri_pths = []
trial_types = []
trial_ids = []

for fmri_pth in fmri_pths:
    fmri_data = nib.load(fmri_pth)

    fmris = 0
    for index, row in trialinfo.iterrows():
        trial_data = fmri_data.slicer[..., row['onset']:row['onset']+row['duration']]
        trial_type = row['trial_type']
        trial_id = trial_type_ids[trial_type]
        subject = re.findall('sub-\d\d', fmri_pth)[0].split('sub-')[-1]
        run = re.findall('run-\d', fmri_pth)[0].split('run-')[-1]

        subject_run_dir = opj(OUTPUT_DIR, f'sub-{subject}', f'run-{run}')
        os.makedirs(subject_run_dir, exist_ok=True)
        
        ext_frmi_pth = opj(subject_run_dir, f'{fmris}_{trial_type}_fmri.nii')
        nib.save(trial_data, ext_frmi_pth)
        
        subjects.append(subject)
        runs.append(run)
        trial_ids.append(trial_id)
        trial_types.append(trial_type)
        ext_fmri_pths.append(ext_frmi_pth)
        
        fmris += 1

label_dict = {
    'subjects': subjects,
    'runs': runs,
    'trial_ids': trial_ids,
    'trial_types': trial_types,
    'ext_frmi_pths': ext_fmri_pths
}
label_df = pd.DataFrame.from_dict(label_dict)
label_df.to_csv(LABEL_CSV, sep=';')